# Données

In [1]:
import os
import requests
from scipy.io import loadmat
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import OneHotEncoder
import numpy as np

def get_mnist(): 
    mnist = fetch_openml('mnist_784')
    X = mnist['data'].values
    X = (mnist['data'] > 128).astype(int)
    y = mnist['target'].values
    y = OneHotEncoder().fit_transform(y.reshape(-1, 1)).toarray()

    return X, y
    
def get_alphadigs():
    if os.path.exists('data/alphadigs.mat'):
        return loadmat('data/alphadigs.mat')
    
    alphadigs_url = 'https://cs.nyu.edu/~roweis/data/binaryalphadigs.mat'
    r = requests.get(alphadigs_url, allow_redirects=True)

    with open('data/alphadigs.mat', 'wb') as f:
        f.write(r.content)
                
    return loadmat('data/alphadigs.mat')
      
def lire_alpha_digit(list_digits) : 
    X = get_alphadigs()
    indices = np.where(np.isin(X["classlabels"], list_digits))[1]
    rows = X["dat"][indices]
    return rows  

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [ ]:
f = lire_alpha_digit(['0', '2', 'P','F', 'Z'])
x,y = get_mnist()

In [7]:
import numpy as np

def relu(x):
    return np.maximum(x, 0)
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Fonctions élémentaires

## Construction d’un RBM et test sur Binary AlphaDigits

In [11]:
import numpy as np
import matplotlib.pyplot as plt

class RBM:
    def __init__(self, p, q):
        self.W = np.random.normal(0, 0.1, (p, q))
        self.a = np.zeros(p)
        self.b = np.zeros(q)

def init_RBM(p, q):
    return RBM(p, q)

def sortie_entree_RBM(RBM, data):
  a = RBM.a.reshape(-1, 1)
  W = RBM.W
  n, p  = data.shape

  input_RBM = 1/(1 + np.exp( - a - np.dot(W, data)))

  return input_RBM

def entree_sortie_RBM(RBM, data):
  b = RBM.b.reshape(-1,1)
  W = RBM.W
  n, p = data.shape

  output_RBM = 1/(1 + np.exp(- b - np.dot(W,data )))

  return output_RBM

def train_RBM(RBM, X, n_hidden, n_iter, learning_rate, batch_size):
        
    n = X.shape[0]
    p = X.shape[1]
    q = n_hidden
    
    X_copy = X.copy()

    for epoch in range(epochs):
        error = 0
        for i in range(0, len(X), batch_size):
            x = X_copy[i:i + batch_size]
            
            v0 = x
            ph_v0 = entree_sortie_RBM(rbm,v0)
            h0 = (ph_v0 > np.random.rand(v0.shape[0], q)).astype(int)
            
            pv_h0 = sortie_entree_RBM(rbm,h0)
            v1 = (pv_h0 > np.random.rand(batch_size, p)).astype(int)
            
            ph_v1 = entree_sortie_RBM (rbm,v1)
            h1 = (ph_v1 > np.random.rand(batch_size, q)).astype(int)
        
            rbm.W += learning_rate * np.dot(v0.T, h0) - np.dot(v1.T, h1)
            rbm.a += learning_rate * np.sum(v0 - v1, axis=0)
            rbm.b += learning_rate * np.sum(ph_v0 - ph_v1, axis=0)
            
            error += np.mean((v0 - v1)**2)
        print("Epoch:", epoch, "Error:", error)
    return rbm


def generer_images_RBM(rbm, iterations, num_images):
    hidden_probs = np.random.rand(num_images, rbm.W.shape[1])
    hidden_states = hidden_probs > 0.5

    for i in range(iterations):
        visible_probs = sortie_entree_RBM(rbm, hidden_states)
        visible_states = visible_probs > np.random.rand(*visible_probs.shape)
        hidden_probs = entree_sortie_RBM (rbm, visible_states)
        hidden_states = hidden_probs > np.random.rand(*hidden_probs.shape)

    return visible_states

def afficher_images(images, n_cols=None):
    n_images, n_pixels = images.shape
    if n_cols is None:
        n_cols = int(np.ceil(np.sqrt(n_images)))
        
        
X = lire_alpha_digit(['0', '2', 'P','F', 'Z'])

# Initialize an RBM with 784 visible units and 100 hidden units
p, q = X.shape[1], 100
rbm = init_RBM(p, q)

# Train the RBM on the MNIST data for 20 epochs with a batch size of 100
k = train_RBM(rbm, X, epochs=20, learning_rate=0.1, batch_size=100)

# Generate 25 new images using the RBM
generated_images = generer_image_RBM(rbm, 100, 25)

# Display the generated images
afficher_images(generated_images)

ValueError: shapes (39,100) and (5,39) not aligned: 100 (dim 1) != 5 (dim 0)